# Proyecto: Reconocimiento de Lenguaje de Señas Americano (ASL)

## Notebook 03: Preprocesamiento de Datos

**Objetivo:** Preparar los datos de manera óptima para el entrenamiento de modelos profundos, incluyendo estrategias avanzadas de aumento de datos que simulen condiciones de webcam real.

**Contenido:**
1. Normalización y estandarización
2. Data Augmentation robusto (rotación, zoom, brillo, contraste)
3. Técnicas de balanceo de clases
4. Generación de conjuntos de validación
5. Preprocesamiento específico para Transfer Learning
6. Guardado de datasets procesados

---

## 0. Importaciones y Configuración

In [ ]:
import os
import sys
import json
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow Version: {tf.__version__}")
print(f"Numpy Version: {np.__version__}")

## 1. Carga de Datos

In [ ]:
# Detectar entorno
try:
    from google.colab import files
    IN_COLAB = True
    BASE_DIR = '/content/sign_language_project'
except:
    IN_COLAB = False
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

# Rutas
DATA_RAW = os.path.join(BASE_DIR, 'data', 'raw')
DATA_PROCESSED = os.path.join(BASE_DIR, 'data', 'processed')
FIGURES_DIR = os.path.join(BASE_DIR, 'results', 'figures')

# Cargar datos
train_df = pd.read_csv(os.path.join(DATA_RAW, 'sign_mnist_train.csv'))
test_df = pd.read_csv(os.path.join(DATA_RAW, 'sign_mnist_test.csv'))

# Separar features y labels
X_train_raw = train_df.drop('label', axis=1).values
y_train_raw = train_df['label'].values
X_test_raw = test_df.drop('label', axis=1).values
y_test_raw = test_df['label'].values

print(f"Datos de entrenamiento: {X_train_raw.shape}")
print(f"Datos de prueba: {X_test_raw.shape}")

# Mapeo de etiquetas
label_to_letter = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I',
    9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P', 15: 'Q', 16: 'R',
    17: 'S', 18: 'T', 19: 'U', 20: 'V', 21: 'W', 22: 'X', 23: 'Y'
}

## 2. Normalización de Datos

In [ ]:
# Normalización [0, 1] - estándar para imágenes
X_train_norm = X_train_raw / 255.0
X_test_norm = X_test_raw / 255.0

# Reshape a formato de imagen (28, 28, 1)
X_train = X_train_norm.reshape(-1, 28, 28, 1)
X_test = X_test_norm.reshape(-1, 28, 28, 1)

# One-Hot Encoding de etiquetas
y_train = to_categorical(y_train_raw, num_classes=24)
y_test = to_categorical(y_test_raw, num_classes=24)

print(f"X_train normalizado: {X_train.shape}, rango: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"y_train one-hot: {y_train.shape}")
print(f"\nDatos listos para entrenamiento de CNN")

## 3. Creación de Conjunto de Validación

In [ ]:
# Dividir training en train y validation (80-20)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42,
    stratify=y_train_raw  # Mantener proporción de clases
)

print(f"Training set: {X_train_split.shape[0]:,} muestras")
print(f"Validation set: {X_val.shape[0]:,} muestras")
print(f"Test set: {X_test.shape[0]:,} muestras")
print(f"\nProporción: {X_train_split.shape[0]/(X_train_split.shape[0]+X_val.shape[0])*100:.1f}% - {X_val.shape[0]/(X_train_split.shape[0]+X_val.shape[0])*100:.1f}%")

## 4. Data Augmentation Avanzado (Crítico para Webcam)

### 4.1 Configuración de ImageDataGenerator con Augmentación Robusta

In [ ]:
# CONFIGURACIÓN CLAVE: Simular condiciones de webcam real
train_datagen = ImageDataGenerator(
    rotation_range=25,              # Rotación hasta ±25° (webcam no está siempre nivelada)
    zoom_range=0.25,                # Zoom de ±25% (distancia variable a cámara)
    width_shift_range=0.2,          # Desplazamiento horizontal
    height_shift_range=0.2,         # Desplazamiento vertical
    shear_range=0.15,               # Cizallamiento (ángulo de cámara)
    brightness_range=[0.5, 1.5],    # CRÍTICO: Variación de brillo (diferentes iluminaciones)
    channel_shift_range=30,         # Cambio de intensidad de canales
    fill_mode='nearest',            # Rellenar con píxeles cercanos
    horizontal_flip=False,          # NO voltear (cambiaría el significado de la seña)
    vertical_flip=False
)

# Validación y test SIN augmentación
val_test_datagen = ImageDataGenerator()

# Fit en datos de entrenamiento
train_datagen.fit(X_train_split)
val_test_datagen.fit(X_val)

print("Data augmentation configurado exitosamente")
print("\nTransformaciones aplicadas:")
print("  - Rotación aleatoria: ±25°")
print("  - Zoom aleatorio: ±25%")
print("  - Desplazamientos: ±20%")
print("  - Brillo: 50% a 150% (CLAVE para webcam)")
print("  - Cizallamiento: ±15°")

### 4.2 Visualización de Augmentación

In [ ]:
# Visualizar efecto del data augmentation
fig, axes = plt.subplots(4, 8, figsize=(20, 10))

# Seleccionar 4 imágenes aleatorias
for row in range(4):
    idx = np.random.randint(0, len(X_train_split))
    sample = X_train_split[idx:idx+1]
    label_idx = np.argmax(y_train_split[idx])
    letter = label_to_letter[label_idx]
    
    # Imagen original
    axes[row, 0].imshow(sample[0, :, :, 0], cmap='gray')
    axes[row, 0].set_title(f'Original: {letter}', fontweight='bold', fontsize=11)
    axes[row, 0].axis('off')
    axes[row, 0].set_facecolor('lightblue')
    
    # 7 versiones aumentadas
    aug_iter = train_datagen.flow(sample, batch_size=1)
    for col in range(1, 8):
        aug_img = next(aug_iter)[0, :, :, 0]
        axes[row, col].imshow(aug_img, cmap='gray')
        axes[row, col].set_title(f'Aug {col}', fontsize=10)
        axes[row, col].axis('off')

plt.suptitle('Ejemplos de Data Augmentation Aplicado', fontsize=18, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03_data_augmentation_examples.png'), dpi=200, bbox_inches='tight')
plt.show()

print("\nEl augmentation simula:")
print("  - Diferentes ángulos de cámara")
print("  - Variaciones de distancia a la cámara")
print("  - Condiciones de iluminación variables (oficina, casa, exterior)")
print("  - Posición variable de la mano en el frame")

### 4.3 Augmentación Adicional Personalizada

In [ ]:
def apply_advanced_augmentation(image, intensity=0.3):
    """
    Aplica transformaciones adicionales no disponibles en ImageDataGenerator.
    Especialmente útil para simular condiciones de webcam real.
    """
    img = image.copy().reshape(28, 28)
    
    # Aplicar transformaciones aleatorias
    if np.random.random() < intensity:
        # 1. Ruido gaussiano (sensor de cámara)
        noise = np.random.normal(0, 0.05, img.shape)
        img = np.clip(img + noise, 0, 1)
    
    if np.random.random() < intensity:
        # 2. Blur (desenfoque de movimiento)
        kernel_size = np.random.choice([3, 5])
        img = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
    if np.random.random() < intensity:
        # 3. Ajuste de contraste
        factor = np.random.uniform(0.7, 1.3)
        img = np.clip((img - 0.5) * factor + 0.5, 0, 1)
    
    if np.random.random() < intensity:
        # 4. Ajuste de gamma (simular diferentes exposiciones)
        gamma = np.random.uniform(0.7, 1.3)
        img = np.power(img, gamma)
    
    return img.reshape(28, 28, 1)

# Demostración
fig, axes = plt.subplots(2, 6, figsize=(18, 6))

for i in range(2):
    idx = np.random.randint(0, len(X_train_split))
    original = X_train_split[idx]
    
    axes[i, 0].imshow(original[:, :, 0], cmap='gray')
    axes[i, 0].set_title('Original', fontweight='bold')
    axes[i, 0].axis('off')
    
    for j in range(1, 6):
        augmented = apply_advanced_augmentation(original, intensity=0.5)
        axes[i, j].imshow(augmented[:, :, 0], cmap='gray')
        axes[i, j].set_title(f'Aug {j}')
        axes[i, j].axis('off')

plt.suptitle('Augmentación Avanzada Personalizada\n(Ruido, Blur, Contraste, Gamma)', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03_advanced_augmentation.png'), dpi=200, bbox_inches='tight')
plt.show()

## 5. Cálculo de Pesos de Clase (Class Weights)

In [ ]:
# Calcular pesos de clase para manejar cualquier desbalance
class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train_raw),
    y=y_train_raw
)

class_weights = dict(enumerate(class_weights_array))

# Visualizar
plt.figure(figsize=(16, 6))
letters = [label_to_letter[i] for i in range(24)]
plt.bar(letters, class_weights_array, color='teal', alpha=0.7, edgecolor='black')
plt.axhline(1.0, color='red', linestyle='--', linewidth=2, label='Peso = 1.0')
plt.title('Pesos de Clase para Balanceo', fontsize=15, fontweight='bold')
plt.xlabel('Letra', fontsize=12)
plt.ylabel('Peso', fontsize=12)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03_class_weights.png'), dpi=200, bbox_inches='tight')
plt.show()

print("Pesos de clase calculados:")
for i, weight in class_weights.items():
    print(f"  {label_to_letter[i]}: {weight:.4f}")

## 6. Preprocesamiento para Transfer Learning

In [ ]:
# Para modelos preentrenados que esperan imágenes RGB
def prepare_for_transfer_learning(X, target_size=(224, 224)):
    """
    Prepara imágenes para modelos pre-entrenados:
    1. Convierte grayscale a RGB (3 canales)
    2. Redimensiona a tamaño requerido (e.g., 224x224)
    """
    # Convertir a RGB repitiendo el canal
    X_rgb = np.repeat(X, 3, axis=-1)
    
    # Redimensionar si es necesario
    if X.shape[1:3] != target_size:
        X_resized = np.array([
            cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
            for img in X_rgb
        ])
        return X_resized
    
    return X_rgb

# Preparar versiones para diferentes modelos
print("Preparando datos para Transfer Learning...")

# Para EfficientNet, ResNet, VGG, etc. (224x224)
X_train_tl_224 = prepare_for_transfer_learning(X_train_split, target_size=(224, 224))
X_val_tl_224 = prepare_for_transfer_learning(X_val, target_size=(224, 224))
X_test_tl_224 = prepare_for_transfer_learning(X_test, target_size=(224, 224))

print(f"\nDatos para Transfer Learning (224x224):")
print(f"  Train: {X_train_tl_224.shape}")
print(f"  Val: {X_val_tl_224.shape}")
print(f"  Test: {X_test_tl_224.shape}")

# Visualizar comparación
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

idx = np.random.randint(0, len(X_train_split))

# Original 28x28
axes[0, 0].imshow(X_train_split[idx, :, :, 0], cmap='gray')
axes[0, 0].set_title('Original 28x28 (Grayscale)', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# RGB 28x28
X_rgb_28 = np.repeat(X_train_split[idx:idx+1], 3, axis=-1)
axes[0, 1].imshow(X_rgb_28[0])
axes[0, 1].set_title('RGB 28x28', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

# RGB 224x224
axes[0, 2].imshow(X_train_tl_224[idx])
axes[0, 2].set_title('RGB 224x224 (Transfer Learning)', fontsize=12, fontweight='bold')
axes[0, 2].axis('off')

# Segunda muestra
idx2 = np.random.randint(0, len(X_train_split))

axes[1, 0].imshow(X_train_split[idx2, :, :, 0], cmap='gray')
axes[1, 0].set_title('Original 28x28 (Grayscale)', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

X_rgb_28_2 = np.repeat(X_train_split[idx2:idx2+1], 3, axis=-1)
axes[1, 1].imshow(X_rgb_28_2[0])
axes[1, 1].set_title('RGB 28x28', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(X_train_tl_224[idx2])
axes[1, 2].set_title('RGB 224x224 (Transfer Learning)', fontsize=12, fontweight='bold')
axes[1, 2].axis('off')

plt.suptitle('Comparación de Formatos de Datos', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '03_formato_transfer_learning.png'), dpi=200, bbox_inches='tight')
plt.show()

## 7. Guardado de Datos Procesados

In [ ]:
# Guardar datos procesados en formato .npz (comprimido)
print("Guardando datos procesados...")

# Dataset estándar (28x28, 1 canal)
np.savez_compressed(
    os.path.join(DATA_PROCESSED, 'data_standard.npz'),
    X_train=X_train_split,
    X_val=X_val,
    X_test=X_test,
    y_train=y_train_split,
    y_val=y_val,
    y_test=y_test,
    y_train_raw=y_train_raw[train_test_split(range(len(y_train_raw)), test_size=0.2, random_state=42, stratify=y_train_raw)[0]],
    y_val_raw=y_train_raw[train_test_split(range(len(y_train_raw)), test_size=0.2, random_state=42, stratify=y_train_raw)[1]],
    y_test_raw=y_test_raw
)

print(f"  - Datos estándar guardados: data_standard.npz")

# Dataset para Transfer Learning (224x224, 3 canales)
np.savez_compressed(
    os.path.join(DATA_PROCESSED, 'data_transfer_learning_224.npz'),
    X_train=X_train_tl_224,
    X_val=X_val_tl_224,
    X_test=X_test_tl_224,
    y_train=y_train_split,
    y_val=y_val,
    y_test=y_test
)

print(f"  - Datos Transfer Learning guardados: data_transfer_learning_224.npz")

# Guardar metadatos de preprocesamiento
preprocessing_metadata = {
    'normalization': 'MinMax [0, 1]',
    'train_samples': int(X_train_split.shape[0]),
    'val_samples': int(X_val.shape[0]),
    'test_samples': int(X_test.shape[0]),
    'train_val_split_ratio': 0.8,
    'augmentation': {
        'rotation_range': 25,
        'zoom_range': 0.25,
        'width_shift_range': 0.2,
        'height_shift_range': 0.2,
        'brightness_range': [0.5, 1.5],
        'shear_range': 0.15
    },
    'class_weights': {label_to_letter[k]: float(v) for k, v in class_weights.items()},
    'label_mapping': label_to_letter
}

with open(os.path.join(DATA_PROCESSED, 'preprocessing_metadata.json'), 'w') as f:
    json.dump(preprocessing_metadata, f, indent=4)

print(f"  - Metadatos guardados: preprocessing_metadata.json")
print("\nTodos los datos procesados guardados exitosamente")

## 8. Resumen del Preprocesamiento

In [ ]:
resumen = f"""
{'='*80}
RESUMEN DE PREPROCESAMIENTO
{'='*80}

1. NORMALIZACIÓN
   - Método: MinMax [0, 1]
   - Formato: (altura, ancho, canales)
   
2. DIVISIÓN DE DATOS
   - Training: {X_train_split.shape[0]:,} muestras ({X_train_split.shape[0]/(X_train_split.shape[0]+X_val.shape[0])*100:.1f}%)
   - Validation: {X_val.shape[0]:,} muestras ({X_val.shape[0]/(X_train_split.shape[0]+X_val.shape[0])*100:.1f}%)
   - Test: {X_test.shape[0]:,} muestras
   - Estratificado: Sí (mantiene proporción de clases)

3. DATA AUGMENTATION
   - Rotación: ±25°
   - Zoom: ±25%
   - Desplazamiento: ±20%
   - Brillo: 50% a 150% (crítico para webcam)
   - Cizallamiento: ±15°
   - Augmentación adicional: Ruido, blur, contraste, gamma

4. FORMATOS PREPARADOS
   - Estándar CNN: (28, 28, 1) - {X_train_split.nbytes / (1024**2):.1f} MB
   - Transfer Learning: (224, 224, 3) - {X_train_tl_224.nbytes / (1024**2):.1f} MB

5. BALANCEO
   - Pesos de clase calculados
   - Rango de pesos: [{min(class_weights_array):.4f}, {max(class_weights_array):.4f}]

6. ARCHIVOS GENERADOS
   - data_standard.npz
   - data_transfer_learning_224.npz
   - preprocessing_metadata.json

{'='*80}

PRÓXIMO PASO:
Notebook 04 - Entrenamiento y comparación de múltiples modelos

{'='*80}
"""

print(resumen)